
---
Author: Anastasios Tzanidakis

In this notebook we report the analysis of PLasTiCC light curves (RRL and EB's) to determine the optimal configuration for Rubuin light curves. Specifically, here are the questions we're trying to understand and optimize:
- What version of LSP should we use: `MultiBand` or `SingleBand`
    - What LSP algorithm should we use: `Fast` or `Floating Mean/generalized`
    - What fourier/base component maximizes the correctly identified periods?
    

- Should we use a pre-determined frequency grid, automatic frequency grid, or zooming-peak approach
    - Pre-determined frequency bin
    - Automatic frequency grid based on light curve duration, oversampling, and nyquist correction
    - Two stage frequency search grid: [highlighted here](https://jakevdp.github.io/blog/2015/06/13/lomb-scargle-in-python/#:~:text=a%20built%2Din%20two%2Dstage%20grid%2Dsearch%20that%20accurately%20determines%20the%20best%20period%20in%20a%20specified%20range)

- How long do these algorithms take in each configuration
    - Timing in bins of number of total observations
    - Timing in bins of number of base components/Fourier terms 
    - Timing in bins frequency grid approach
   
   
**Extra Analysis**/future TODO:
- What is the **minimum** number of observatiosn we need per light curve in order to recover the correct period?
- After $\mathcal{\tau}$ years of the survey (PlasTiCC maximum is 3 years; supposed Rubin DR3) - how **many** correctly identified periods would we find?
- Should the light curve moment also capture more information from the power spectrum (i.e TOP N peaks?). How many more correctly identified periodicities would be enough?
- Using the Rubin `Opsims` cadence, can there be an optimized pre-determined frequency window that maximizes period finding while minimizing computation?

## Section 1 - Metrics & Setp-up



### PlasTiCC Light Curves & Sample
--- 

For this analysis, we will make a few assumptions about the light curves under investigation: 
- Consider a sample of only RR Lyrae (RRL) and Eclipsing Binaries (EB)
- Investigate light curves with a duration (T) less than 365 days
- To mimic the effects of forced-photometry, we will include in our analysis both `detection==0, 1` (both detections & non-detections) in the PlasTiCC light curves
- We set a minimum number of observations per light curve under investigation to be $N_{min}$=10


### Period Metric
--- 
We would like to define some success metric for the correctly identified periods within some margin of error. For my metric, I will estimate a simple percentage error:
`
$$\begin{equation} Err = \frac{|p_{inj}-p_{rec}|}{p_{inj}}\end{equation}$$

where $p_{inj}$ is the injected true period of the light curve, while the $p_{rec}$ is the recovered period from the Lomb-Scargle Periodogram. We note that for this analysis we will assume that the **correct period corresponds to the maximum power**, without considering other harmonics that might be caused by the survey cadence.

In [87]:
import numpy as np
from astropy.io import ascii
import os 
from tqdm import tqdm
from astropy.time import Time
import pandas as pd
import time
import random
from gatspy import periodic, datasets
import matplotlib.pyplot as plt
import matplotlib
from astropy.table import Table
from gatspy import datasets, periodic
import gatspy
import scipy.stats as sci_stat
import sys

%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams['savefig.dpi'] = 250
rcParams['font.size'] = 20

global data_path
data_path = '../data/plasticc/data/'

# Caution, be careful when ignoring warnings!
import sys
import warnings

if not sys.warnoptions:
    """Not reccomended, but for the moment supress warnings!"""
    warnings.simplefilter("ignore")

In [81]:
# TODO: Add entire PlasTiCC stream?!
data = pd.read_csv(data_path + "plasticc_test_lightcurves_11.csv.gz",
                  compression='gzip',
                  error_bad_lines=False)

meta_info = ascii.read(data_path + "plasticc_train_metadata.csv") # ascii meta since it's smaller
meta_theta_EB = ascii.read(data_path + 'plasticc_modelpar/' + 'plasticc_modelpar_016_EB.csv')
meta_theta_RRL = ascii.read(data_path + 'plasticc_modelpar/' + 'plasticc_modelpar_092_RRL.csv')

In [94]:
# Define some helper functions for the analysis

def generate_lc(obj_id, band='all', data_table=data, det='all', dt_cut=365):
    """Unpack and return PlastiCC data in numpy array format.

    Paremters
    ---------
    obj_id: Object ID
    band: Photometric bandpass filter. 'all' includes ugrizy, or 'ugrizy'
    data_table: Pandas data table containing the light curves
    det: Detection from the image subtraction algorithm. ==1 detection, ==0 not detection (i.e upper limit) or 'all': uses both 0 & 1
    dt_cut: Light curve duration cut in days (default 365 days)
    Returns
    -------
    mjd, magnitude, magnitude_error, filter (if band=='all')
    """

    if det==0 or det==1:
        data_table_mod = data_table[data_table['detected_bool']==det]
    elif det=='all':
        data_table_mod = data_table # select both

    # Select light curve based on the ID
    lc = data_table_mod[data_table_mod['object_id']==obj_id]

    lsst_bands = list('ugrizy') # lsst photomeric bands

    lc_array = lc.to_numpy()

    # Capture empty light curve
    assert len(lc_array[:,1])>0, ("Sorry, it seems like your obj_id query was wrong!")

    mjd, flux, flux_err = lc_array[:,1], lc_array[:,3], lc_array[:,4]
    flt = lc_array[:,2].astype(int).astype(str)

    for j in range(6):
        flt[flt==str(j)] = lsst_bands[j]

    # Baseline cut
    baseline = mjd-mjd[0] # calculate baseline
    mjd, flux, flux_err, flt = mjd[baseline<=dt_cut], flux[baseline<=dt_cut], flux_err[baseline<=dt_cut], flt[baseline<=dt_cut]

    if band=='all':
        return mjd, flux, flux_err, flt
    else:
        return mjd[flt==band], flux[flt==band], flux_err[flt==band], flt[flt==band]

def generate_toi_table(data, meta_info, meta_theta_EB, meta_theta_RRL):
    """
    Generate table that contains the light curve ID and transient type given a number of observations cut.

    Input
    -----
    data: Head data table that contains photometry
    meta_info: Table that contains the meta-data (i.e classification name)
    meta_theta_<TYPE>: Table that contains metadata information (i.e Period)
    nthresh (int): Minimum number of observations in each band must contain
    """
    id_av_rrl, id_av_eb = [], []

    for uid in tqdm(np.unique(data['object_id'])):
        ww = np.where(meta_theta_EB['object_id'] == uid)
        if np.shape(ww)[-1]==1:
            id_av_eb.append(uid)

    for uid in tqdm(np.unique(data['object_id'])):
        ww = np.where(meta_theta_RRL['object_id'] == uid)
        if np.shape(ww)[-1]==1:
            id_av_rrl.append(uid)

    id_av_rrl, id_av_eb = np.array(id_av_rrl), np.array(id_av_eb)

    _id1 = np.array(['rrl' for _ in range(len(id_av_rrl))])
    _id2 = np.array(['eb' for _ in range(len(id_av_eb))])

    # All ID's and & ID tags
    all_id = np.concatenate([id_av_rrl, id_av_eb])
    _id_all = np.concatenate([_id1, _id2])
            
    # Final TOI table
    toi_table = Table([all_id, _id_all], names=('obj_id', 'type'))

    return toi_table

In [96]:
# Fetch the Object_id of all the targets of interest (toi)
toi_table = generate_toi_table(data, meta_info, meta_theta_EB, meta_theta_RRL)

100%|██████████| 345996/345996 [00:24<00:00, 13974.97it/s]


In [88]:
# Define some additional helper functions for metadata handiling and processing
def fetch_meta_info(lc_id, lc_type):
    """Fetch metadata for transient type.

    Input
    -----
    lc_id: Light curve ID
    lc_type: classification type (i.e rrl, eb)

    Output
    ------
    meta_<type>_table: Table that contains metadata (i.e period and other physical properties)
    """
    if lc_type=='rrl':
        # crossmatch to approprirate table
        xm_ = np.where(meta_theta_RRL['object_id']==lc_id)
        return meta_theta_RRL[xm_]
    elif lc_type=='eb':
        # crossmatch to approprirate table
        xm_ = np.where(meta_theta_EB['object_id']==lc_id)
        return meta_theta_EB[xm_]

# Write a function that will generate N random from each class (equal)
def draw_rand_trans(table, N, class_type='rrl'):
    """Draw N random object_id's from the specific class (with no repeats)
       Note: It will not draw the same transiennt
    """
    # isolate each unique class
    req_tab = table[table['type']==class_type]

    # Random number generator w/o repeat
    rng = np.random.default_rng()
    rn = rng.choice(len(req_tab), size=N, replace=False)

    return req_tab[rn]

def percent_error(true, obs):
    """Calculate the absolute percent error."""
    return abs(true-obs)/true

## Section 2.1 - Adressing the Single Band LSP
We begin our analysis on single band photometric LSP analysis. Here we will be adressing a few questions: 

- For each photometric filter, what configuration minimizes the percent error?
    - Which method is better, `Fast` or `Floating Mean`
    - What frequency grid to choose?

- Timing analysis: 
    - Timing per number of total observations (likely in bins=10)
    - Timing per number of components (only applies to Floating Mean)
    - Timing per grid search
    
    
    
**Internal Notes/Figures**:
- Start with with `Fast`:
    - What frequency grid and configuration

In [109]:
# Generate a sample of RRL & EB's with 10 total observations per photometric band
rrl_table = draw_rand_trans(toi_table, 5_000, 'rrl')
eb_table = draw_rand_trans(toi_table, 5_000, 'eb')

def band_cut(dat_table, band='r', nthresh=10, dt_cut=365):
    """Return object_id to sources on a specified photometric filter given some threshold detection cut."""
    
    idi = []  
    for _id in tqdm(dat_table['obj_id']):
        # Fetch light curve info
        lc = generate_lc(_id, band=band, det='all', dt_cut=dt_cut)
        if len(lc[0])>=nthresh:
            idi.append(_id)
    return np.array(idi)

rrl_u, eb_u = band_cut(rrl_table, band='u', nthresh=10), band_cut(eb_table, band='u', nthresh=10)

100%|██████████| 5000/5000 [01:10<00:00, 70.43it/s]


In [107]:
len(rrl_u), len(eb_u)

100%|██████████| 2000/2000 [00:27<00:00, 71.51it/s]


In [108]:
len(ll)

67